In [24]:
df.columns

Index(['Unnamed: 0', 'KNN', 'Sampling', 'Normalisasi', 'Jarak', 'Nilai K',
       'Time Train', 'Score Time', 'Akurasi', 'Presisi', 'Recall', 'F1 Score',
       'True SD Pred SD', 'True SD Pred SMP', 'True SD Pred SMA',
       'True SMP Pred SD', 'True SMP Pred SMP', 'True SMP Pred SMA',
       'True SMA Pred SD', 'True SMA Pred SMP', 'True SMA Pred SMA'],
      dtype='object')

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
df=pd.read_excel("Rekap Dataset 03032020.xlsx")
df.columns
features=df[[
       'Selisih Isi', 'Jumlah Chapter', 'Font Size', 'Spasi',
       'paragraf', 'jumlah kalimat', 'jumlah total kata', 'jumlah jenis kata',
       'total suku kata', 'jumlah suku kata non poly', 'jumlah suku kata poly',
       'romantis', 'realistis', 'khayalan', 'general', 'kata/kalimat',
       'kata/paragraf', 'kata/halaman', 'kalimat/paragraf', 'kalimat/halaman',
       'paragraf/halaman', 'khayalan/kata', 'romantik/kata', 'realistik/kata',
       'general/kata']]
label=df['label']

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import neighbors,model_selection
from sklearn.model_selection import  cross_validate,cross_val_predict
from sklearn.metrics import confusion_matrix,classification_report,make_scorer, accuracy_score, precision_score, recall_score, f1_score

from sklearn import preprocessing

from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=100)
import warnings
warnings.filterwarnings('ignore')

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score,average='macro'),
           'recall' : make_scorer(recall_score,average='macro'), 
           'f1_score' : make_scorer(f1_score,average='macro')}
listnilai=list(range(1,2))
pengukuranjarak=["euclidean", "manhattan","chebyshev","minkowski 1.5"]
jenisnormalisasi=["Tanpa Normalisasi","MinMaxScaler","MaxAbsScaler","StandardScaler","RobustScaler","Normalizer",
                  "QuantileTransformer uniform","QuantileTransformer normal","PowerTransformer yeo-johnson"]
#jenisnormalisasi=["Data Asli","MinMaxScaler","StandardScaler"]
jenisresampling=["Tanpa Resampling","SMOTE"]
jenisknn=["KNN","WDKNN"]
data=[]

for jenisklasifikasi in jenisknn:
    if jenisklasifikasi=="KNN":
        weights="uniform"
    elif jenisklasifikasi=="WDKNN":
        weights="distance"
    for normalisasi in jenisnormalisasi:
        X=features.loc[:,:]
        y=label.loc[:]
        if(normalisasi=="Data Asli"):
            X=X
        elif(normalisasi=="MinMaxScaler"):    
            X=preprocessing.MinMaxScaler().fit_transform(X)
        elif(normalisasi=="MaxAbsScaler"): 
            X=preprocessing.MaxAbsScaler().fit_transform(X)
        elif(normalisasi=="StandardScaler"): 
            X=preprocessing.StandardScaler().fit_transform(X)
        elif(normalisasi=="RobustScaler"): 
            X=preprocessing.RobustScaler().fit_transform(X)
        elif(normalisasi=="Normalizer"): 
            X=preprocessing.Normalizer().fit_transform(X)
        elif(normalisasi=="QuantileTransformer uniform"): 
            X=preprocessing.QuantileTransformer(output_distribution='uniform').fit_transform(X)
        elif(normalisasi=="QuantileTransformer normal"): 
            X=preprocessing.QuantileTransformer(output_distribution='normal').fit_transform(X)
        elif(normalisasi=="PowerTransformer yeo-johnson"): 
            X=preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(X)
        for resampling in jenisresampling:
            if(resampling=="Tanpa Resampling"):
                X=X
                y=y
            elif(resampling=="SMOTE"):
                X,y=sm.fit_resample(features, label)
            for jarak in pengukuranjarak:
                for nilaik in listnilai:
                    if(jarak=='mahalanobis' or jarak=="seuclidean" or jarak=="wminkowski"):
                        continue
                        #model=neighbors.KNeighborsClassifier(n_neighbors=5,metric=jarak,metric_params={'V': np.cov(X)})
                    elif(jarak=='minkowski 1.5'):
                        model=neighbors.KNeighborsClassifier(n_neighbors=nilaik,metric="minkowski",p=1.5,weights=weights)
                    else:
                        model=neighbors.KNeighborsClassifier(n_neighbors=nilaik,metric=jarak,weights=weights)
                    results = cross_validate(estimator=model,X=X,y=y,cv=10,scoring=scoring)
                    y_pred = cross_val_predict(estimator=model,X=X,y=y,cv=10)
                    unique_label = np.unique([y, y_pred])
                    cmtx = pd.DataFrame(
                        confusion_matrix(y, y_pred, labels=unique_label), 
                        index=['true:{:}'.format(x) for x in unique_label], 
                        columns=['pred:{:}'.format(x) for x in unique_label]
                    )
                    #print(cmtx)
                    #,cmtx.loc['true:SMA','pred:SMA'])
                    #for i in results:
                        #print(i,results[i].mean())
                    print(resampling,normalisasi,jarak,jenisklasifikasi,nilaik,"=",results["test_accuracy"].mean())
                    #print(resampling,normalisasi,jarak,jenisklasifikasi)

                    data.append([jenisklasifikasi,resampling,normalisasi,jarak,nilaik,
                                results["fit_time"].mean(),results["score_time"].mean(),
                                results["test_accuracy"].mean(),results["test_precision"].mean(),
                                results["test_recall"].mean(),results["test_f1_score"].mean(),
                                cmtx.loc['true:SD','pred:SD'],cmtx.loc['true:SD','pred:SMP'],
                                cmtx.loc['true:SD','pred:SMA'],cmtx.loc['true:SMP','pred:SD'],
                                cmtx.loc['true:SMP','pred:SMP'],cmtx.loc['true:SMP','pred:SMA'],
                                cmtx.loc['true:SMA','pred:SD'],cmtx.loc['true:SMA','pred:SMP'],
                                cmtx.loc['true:SMA','pred:SMA']])

Tanpa Resampling Tanpa Normalisasi euclidean KNN 1 = 0.5011627106983144
Tanpa Resampling Tanpa Normalisasi euclidean KNN 2 = 0.5897695218438253
Tanpa Resampling Tanpa Normalisasi euclidean KNN 3 = 0.5537461300309597
Tanpa Resampling Tanpa Normalisasi euclidean KNN 4 = 0.6055538355693154
Tanpa Resampling Tanpa Normalisasi euclidean KNN 5 = 0.5952906776745788
Tanpa Resampling Tanpa Normalisasi euclidean KNN 6 = 0.6105245958032336
Tanpa Resampling Tanpa Normalisasi euclidean KNN 7 = 0.6213433092535261
Tanpa Resampling Tanpa Normalisasi euclidean KNN 8 = 0.6363140694874441
Tanpa Resampling Tanpa Normalisasi euclidean KNN 9 = 0.6172256621947023
Tanpa Resampling Tanpa Normalisasi euclidean KNN 10 = 0.6049054007567939
Tanpa Resampling Tanpa Normalisasi euclidean KNN 11 = 0.6119917440660474
Tanpa Resampling Tanpa Normalisasi euclidean KNN 12 = 0.6210165118679051
Tanpa Resampling Tanpa Normalisasi euclidean KNN 13 = 0.6374544203646371
Tanpa Resampling Tanpa Normalisasi euclidean KNN 14 = 0.6433

Tanpa Resampling Tanpa Normalisasi manhattan KNN 15 = 0.6312796697626418
Tanpa Resampling Tanpa Normalisasi manhattan KNN 16 = 0.6321620227038185
Tanpa Resampling Tanpa Normalisasi manhattan KNN 17 = 0.6371620227038184
Tanpa Resampling Tanpa Normalisasi manhattan KNN 18 = 0.6380443756449949
Tanpa Resampling Tanpa Normalisasi manhattan KNN 19 = 0.637781217750258
Tanpa Resampling Tanpa Normalisasi manhattan KNN 20 = 0.6430443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 21 = 0.6430443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 22 = 0.6430443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 23 = 0.6380443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 24 = 0.6430443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 25 = 0.6380443756449948
Tanpa Resampling Tanpa Normalisasi manhattan KNN 26 = 0.642781217750258
Tanpa Resampling Tanpa Normalisasi manhattan KNN 27 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi manhattan KNN 28 = 

Tanpa Resampling Tanpa Normalisasi chebyshev KNN 29 = 0.6427812177502581
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 30 = 0.6483075335397317
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 31 = 0.6375180598555212
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 32 = 0.6533075335397317
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 33 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 34 = 0.6533075335397317
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 35 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 36 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 37 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 38 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 39 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 40 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 41 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi chebyshev KNN 42 = 0.648

Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 41 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 42 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 43 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 44 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 45 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 46 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 47 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 48 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 49 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 50 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 51 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 52 = 0.648044375644995
Tanpa Resampling Tanpa Normalisasi minkowski 1.5 KNN 53 = 0.648044375644995
Tanpa Resamp

SMOTE Tanpa Normalisasi euclidean KNN 61 = 0.5222222222222224
SMOTE Tanpa Normalisasi euclidean KNN 62 = 0.5254273504273504
SMOTE Tanpa Normalisasi euclidean KNN 63 = 0.5363247863247863
SMOTE Tanpa Normalisasi euclidean KNN 64 = 0.5444444444444444
SMOTE Tanpa Normalisasi euclidean KNN 65 = 0.5339743589743591
SMOTE Tanpa Normalisasi euclidean KNN 66 = 0.5363247863247863
SMOTE Tanpa Normalisasi euclidean KNN 67 = 0.5256410256410257
SMOTE Tanpa Normalisasi euclidean KNN 68 = 0.5228632478632478
SMOTE Tanpa Normalisasi euclidean KNN 69 = 0.5363247863247863
SMOTE Tanpa Normalisasi euclidean KNN 70 = 0.5333333333333333
SMOTE Tanpa Normalisasi euclidean KNN 71 = 0.5361111111111111
SMOTE Tanpa Normalisasi euclidean KNN 72 = 0.5386752136752136
SMOTE Tanpa Normalisasi euclidean KNN 73 = 0.5472222222222222
SMOTE Tanpa Normalisasi euclidean KNN 74 = 0.5305555555555556
SMOTE Tanpa Normalisasi euclidean KNN 75 = 0.5365384615384616
SMOTE Tanpa Normalisasi euclidean KNN 76 = 0.5363247863247863
SMOTE Ta

...

Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 43 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 44 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 45 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 46 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 47 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 48 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 49 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 50 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 51 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 52 = 0.648044375644995
Tanpa Resampling PowerTransformer yeo-johnson minkowski 1.5 WDKNN 53 = 0.648044375644995
Tanpa Resampling Powe

SMOTE PowerTransformer yeo-johnson euclidean WDKNN 42 = 0.6485042735042735
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 43 = 0.6452991452991454
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 44 = 0.6452991452991452
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 45 = 0.6397435897435897
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 46 = 0.6401709401709402
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 47 = 0.6427350427350428
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 48 = 0.6371794871794871
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 49 = 0.6292735042735043
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 50 = 0.6264957264957265
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 51 = 0.632051282051282
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 52 = 0.629059829059829
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 53 = 0.6373931623931625
SMOTE PowerTransformer yeo-johnson euclidean WDKNN 54 = 0.6292735042735043
SMOTE PowerTransformer yeo-

SMOTE PowerTransformer yeo-johnson manhattan WDKNN 53 = 0.6587606837606839
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 54 = 0.6583333333333334
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 55 = 0.6636752136752136
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 56 = 0.6638888888888889
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 57 = 0.6638888888888889
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 58 = 0.6641025641025641
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 59 = 0.6613247863247864
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 60 = 0.663888888888889
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 61 = 0.663888888888889
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 62 = 0.663888888888889
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 63 = 0.6641025641025642
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 64 = 0.6641025641025642
SMOTE PowerTransformer yeo-johnson manhattan WDKNN 65 = 0.6613247863247864
SMOTE PowerTransformer yeo-j

SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 63 = 0.6284188034188036
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 64 = 0.6286324786324787
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 65 = 0.6207264957264957
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 66 = 0.6288461538461539
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 67 = 0.6237179487179488
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 68 = 0.6235042735042735
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 69 = 0.617948717948718
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 70 = 0.6211538461538462
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 71 = 0.620940170940171
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 72 = 0.6262820512820513
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 73 = 0.6262820512820513
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 74 = 0.6235042735042735
SMOTE PowerTransformer yeo-johnson chebyshev WDKNN 75 = 0.6262820512820514
SMOTE PowerTransformer yeo-

SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 69 = 0.6450854700854701
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 70 = 0.6506410256410257
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 71 = 0.6480769230769231
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 72 = 0.650854700854701
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 73 = 0.6480769230769231
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 74 = 0.6504273504273506
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 75 = 0.6448717948717948
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 76 = 0.6448717948717948
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 77 = 0.6395299145299146
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 78 = 0.6423076923076922
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 79 = 0.6395299145299146
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKNN 80 = 0.6367521367521367
SMOTE PowerTransformer yeo-johnson minkowski 1.5 WDKN

In [10]:
import pandas as pd
dataexport=pd.DataFrame(data)

dataexport.columns=["KNN","Sampling",
"Normalisasi",
"Jarak",
"Nilai K",
"Time Train",
"Score Time",
"Akurasi",
"Presisi",
"Recall",
"F1 Score",
"True SD Pred SD",
"True SD Pred SMP",
"True SD Pred SMA",
"True SMP Pred SD",
"True SMP Pred SMP",
"True SMP Pred SMA",
"True SMA Pred SD",
"True SMA Pred SMP",
"True SMA Pred SMA"]
dataexport.to_excel("HasilRevisiSemhas06082020.xlsx")
dataexport.to_json("HasilRevisiSemhas06082020.json",orient='index')

In [2]:
import pandas as pd
import numpy as np

df=pd.read_excel("HasilRevisiSemhas06082020.xlsx")
hasil=df.groupby(["KNN","Sampling","Normalisasi","Jarak"]).agg({'Akurasi': ['mean', 'min', 'max'],'F1 Score':['mean','min', 'max']})
#hasil.to_excel("hasilpengaturanbaru.xlsx")
hasil

Akurasi            \
                                                            mean       min   
KNN   Sampling         Normalisasi       Jarak                               
KNN   SMOTE            MaxAbsScaler      chebyshev      0.539393  0.494872   
                                         euclidean      0.550177  0.506410   
                                         manhattan      0.559733  0.509402   
                                         minkowski 1.5  0.553735  0.507051   
                       MinMaxScaler      chebyshev      0.539393  0.494872   
...                                                          ...       ...   
WDKNN Tanpa Resampling StandardScaler    minkowski 1.5  0.637251  0.556202   
                       Tanpa Normalisasi chebyshev      0.623149  0.482019   
                                         euclidean      0.630008  0.501163   
                                         manhattan      0.630536  0.490344   
                                         minkowski 1.5  0.630147  0.474754   

                                                                  F1 Score  \
                                                             max      mean   
KNN   Sampling         Normalisasi       Jarak                               
KNN   SMOTE            MaxAbsScaler      chebyshev      0.680128  0.537705   
                                         euclidean      0.701282  0.547217   
                                         manhattan      0.730983  0.555436   
                                         minkowski 1.5  0.711752  0.550121   
                       MinMaxScaler      chebyshev      0.680128  0.537705   
...                                                          ...       ...   
WDKNN Tanpa Resampling StandardScaler    minkowski 1.5  0.648044  0.288887   
                       Tanpa Normalisasi chebyshev      0.648044  0.281144   
                                         euclidean      0.648044  0.281611   
                                         manhattan      0.648044  0.279331   
                                         minkowski 1.5  0.648044  0.280940   

                                                                            
                                                             min       max  
KNN   Sampling         Normalisasi       Jarak                              
KNN   SMOTE            MaxAbsScaler      chebyshev      0.496239  0.665874  
                                         euclidean      0.509232  0.684025  
                                         manhattan      0.508673  0.718264  
                                         minkowski 1.5  0.502209  0.697039  
                       MinMaxScaler      chebyshev      0.496239  0.665874  
...                                                          ...       ...  
WDKNN Tanpa Resampling StandardScaler    minkowski 1.5  0.268506  0.374013  
                       Tanpa Normalisasi chebyshev      0.264069  0.374333  
                                         euclidean      0.269238  0.370336  
                                         manhattan      0.265746  0.347655  
                                         minkowski 1.5  0.267338  0.343957  

[144 rows x 6 columns]

In [3]:
hasil=df.groupby(["KNN"]).agg({'Akurasi': ['mean', 'min', 'max'],'F1 Score':['mean','min', 'max']})
hasil.to_excel("hasilknnrevisi.xlsx")
hasil

Akurasi                      F1 Score                    
           mean       min       max      mean       min       max
KNN                                                              
KNN    0.597080  0.458048  0.730983  0.418674  0.262717  0.718264
WDKNN  0.643851  0.458048  0.730983  0.465070  0.264069  0.718264

In [15]:
hasil=df.groupby(["Sampling"]).agg({'Akurasi': ['mean', 'min', 'max'],'F1 Score':['mean','min', 'max']})
#hasil.to_excel("hasilsampling.xlsx")
hasil

Akurasi                      F1 Score                    
                      mean       min       max      mean       min       max
Sampling                                                                    
SMOTE             0.598476  0.494872  0.730983  0.591393  0.496239  0.718264
Tanpa Resampling  0.642456  0.458048  0.712499  0.292350  0.262717  0.483758

In [16]:
hasil=df.groupby(["Jarak"]).agg({'Akurasi': ['mean', 'min', 'max'],'F1 Score':['mean','min', 'max']})
#hasil.to_excel("hasiljarak.xlsx")
hasil

Akurasi                      F1 Score                    
                   mean       min       max      mean       min       max
Jarak                                                                    
chebyshev      0.612137  0.468701  0.701944  0.435483  0.262717  0.665874
euclidean      0.619516  0.501163  0.707236  0.441120  0.266758  0.684025
manhattan      0.627605  0.458048  0.730983  0.447549  0.265746  0.718264
minkowski 1.5  0.622606  0.474754  0.712499  0.443335  0.267338  0.697039

In [17]:
hasil=df.groupby(["Normalisasi"]).agg({'Akurasi': ['mean', 'min', 'max'],'F1 Score':['mean','min', 'max']})
#hasil.to_excel("hasilnormalisasi.xlsx")
hasil

Akurasi                      F1 Score  \
                                  mean       min       max      mean   
Normalisasi                                                            
MaxAbsScaler                  0.619394  0.494872  0.730983  0.439664   
MinMaxScaler                  0.618733  0.494872  0.730983  0.440228   
Normalizer                    0.617772  0.458048  0.730983  0.437848   
PowerTransformer yeo-johnson  0.623529  0.494872  0.730983  0.445480   
QuantileTransformer normal    0.627853  0.494872  0.730983  0.451805   
QuantileTransformer uniform   0.622580  0.468701  0.730983  0.443454   
RobustScaler                  0.620459  0.494872  0.730983  0.442692   
StandardScaler                0.617747  0.494872  0.730983  0.439579   
Tanpa Normalisasi             0.616126  0.474754  0.730983  0.436095   

                                                  
                                   min       max  
Normalisasi                                       
MaxAbsScaler                  0.267829  0.718264  
MinMaxScaler                  0.266758  0.718264  
Normalizer                    0.267588  0.718264  
PowerTransformer yeo-johnson  0.273968  0.718264  
QuantileTransformer normal    0.275866  0.718264  
QuantileTransformer uniform   0.264814  0.718264  
RobustScaler                  0.267421  0.718264  
StandardScaler                0.262717  0.718264  
Tanpa Normalisasi             0.264069  0.718264